# Practice SA on the Wolf-Sheep

This is the second practical exercise for the participants of the workshop on *sensitivity analysis* (SA) at the Social Simulation Festival 2021.

The task remains the same: is to perform a more comperhansive SA than we did previously, but the model is different - [Wolf-Sheep](https://ccl.northwestern.edu/netlogo/models/WolfSheepPredation). Again, explore more parameters, their ranges and add more scenarios. We kept the structure of the notebook as previous, so you can always read something up. Let's go!

## 0. Installations and imports

In [1]:
# Clone the repo to make its file available for Google Colab
!git clone https://github.com/BROSE-Uninc/SSF2021.git

fatal: destination path 'SSF2021' already exists and is not an empty directory.


In [2]:
# Install necessary packages
!pip install ema_workbench mesa ipyparallel SALib &> /dev/null

^C


In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import random

# Import EMA Workbench modules
from ema_workbench import ReplicatorModel, RealParameter, BooleanParameter, IntegerParameter, Constant, TimeSeriesOutcome, perform_experiments, save_results, ema_logging

# Initialize logger to keep track of experiments run
ema_logging.log_to_stderr(ema_logging.INFO)

# Import Mesa Wolf-Sheep model
from SSF2021.wolf_sheep import model

## 1. Load the model

The very first step of SA with EMA Workbench is to define or "load" the model as a function. That is, EMA Workbench treats all models as functions (read *black box*). They are supposed to have **inputs** (parameters, constants, uncertainties and policy levers) and **outputs** (outcomes, KPIs). The model structure is not interesting for EMA Workbench. It may be something simple as `def func(x)` which just returns x + 1.

In [3]:
# setting up the wolf-sheep model simulation as a function
def model_wolf_sheep(height=20,
                     width=20,
                     initial_sheep=100,
                     initial_wolves=50,
                     sheep_reproduce=0.04,
                     wolf_reproduce=0.05,
                     wolf_gain_from_food=20,
                     grass=False,
                     grass_regrowth_time=30,
                     sheep_gain_from_food=4,
                     steps=200):
    
    from SSF2021.wolf_sheep import model
    
    # Initialising the model
    wolf_sheep = model.WolfSheep(height=20,
                                   width=20,
                                   initial_sheep=100,
                                   initial_wolves=50,
                                   sheep_reproduce=0.04,
                                   wolf_reproduce=0.05,
                                   wolf_gain_from_food=20,
                                   grass=False,
                                   grass_regrowth_time=30,
                                   sheep_gain_from_food=4)
                
    # Run the model steps times
    wolf_sheep.run_model(steps)
    
    # Get model outcomes
    outcomes = wolf_sheep.datacollector.get_model_vars_dataframe()
    
    # Return model outcomes
    # below to be changed!
    return {'TIME' : list(range(steps + 1)),
            "Wolves" : outcomes["Wolves"].tolist(),
            "Sheep" : outcomes["Sheep"].tolist()}

Now, let's parameterize and test out our Mesa model. What is supposed to happen? First, we shouldn't get any error 😅. Second, after we run `model_wolf_sheep` function it has to give us a set of model outcomes. Let's try.

In [5]:
# Parametrize the model
height=20
width=20
initial_sheep=100
initial_wolves=50
sheep_reproduce=0.04
wolf_reproduce=0.05
wolf_gain_from_food=20
grass=False
grass_regrowth_time=30
sheep_gain_from_food=4
steps=10

model_wolf_sheep(height=height, 
                 width=width, 
                 initial_sheep=initial_sheep, 
                 initial_wolves=initial_wolves, 
                 sheep_reproduce=sheep_reproduce, 
                 wolf_reproduce=wolf_reproduce, 
                 wolf_gain_from_food=wolf_gain_from_food,
                 grass=grass,
                 grass_regrowth_time=grass_regrowth_time,
                 sheep_gain_from_food=sheep_gain_from_food,
                 steps=steps)

{'TIME': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'Wolves': [50, 52, 53, 55, 58, 54, 53, 57, 60, 63, 62],
 'Sheep': [100, 96, 93, 89, 82, 79, 76, 68, 63, 60, 54]}

## 2. Design experiments

Now it's time to design experiments. What does it mean? Well, we have to specify:

* **which model parameters** aka *inputs* are we going to sample, what are their **ranges**, and random **distributions**,
* what we will keep as **constants** and do not change over the model run,
* and finally which **outcomes** we want to observe.

It's an important step in SA workflow and we have to be careful. Because if parameter ranges are too narrow or they're sampled from e.g. a Normal distribution, there is a chance that you'll overlook import model behavior. This is why model parameters are named **uncertainties** in the EMA Workbench. We often do not know parameter vales and how to explore many plausible options.

Now let's talk about "tech" part. First we have to initialize an instance of EMA Workbench called `ReplicatorModel`. This is how we "connect" EMA Workbench to our Python model. We have to pass a name of our model to `ReplicatorModel`, and also pass the function that we defined previously.

In [6]:
# Instantiate and pass the model 
model = ReplicatorModel(..., function=...)

In [ ]:
# Define model parameters and their ranges to be sampled
model.uncertainties = [RealParameter(..., ..., ...)]

In [ ]:
# Define model parameters that will remain constant
model.constants = [Constant("parameter_A", parameter_A)]

In [ ]:
# Define model outcomes
model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('Wolves'),
                  TimeSeriesOutcome('Sheep')]

In [ ]:
# Define the number of replications
model.replications = 10

## 3. Run the model

In [ ]:
# Run experiments with the aforementioned parameters and outputs
results = perform_experiments(model, 100)

In [ ]:
# Get the results
experiments, outcomes = results

In [ ]:
experiments.head()

In [ ]:
outcomes.keys()

In [ ]:
from ema_workbench.util.utilities import save_results, load_results
import os

In [ ]:
# Creaet a directory to store the results
directory = 'results/wolf_sheep'
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
# Save the results
save_results(results, 'results/wolf_sheep/results.tar.gz')a

In [ ]:
# Load the results
results = load_results('results/wolf_sheep/results.tar.gz')

In [ ]:
experiments, outcomes = results

### A bit of preprocessing

In [ ]:
print(random.choice(list(outcomes)))
outcomes[random.choice(list(outcomes))].shape

In [ ]:
mean_outcomes = {key:np.mean(outcomes[key],axis=1) for key in outcomes.keys()}
mean_results = (experiments.copy(), mean_outcomes)

In [ ]:
# Now the shape of this array doesn't have 10 in it  
mean_outcomes[random.choice(list(outcomes))].shape

### Visuals!

In [ ]:
from ema_workbench.analysis.plotting import lines

In [ ]:
# plotting all of the results
plt.rcParams['figure.figsize'] = [10, 12]

figure = lines(experiments, outcomes_2D) #show lines, and end state density
plt.show()

## 4. Sensitivity analysis

In [ ]:
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SSF2021.src.plot import plot_sobol_indices
sns.set_style('white')

In [ ]:
# Specify the problem
problem = get_SALib_problem(model.uncertainties)

In [ ]:
# Select and normalize an outcome
normalized_outcomes = ...

In [ ]:
# Perform Sobol SA
Si = sobol.analyze(problem=problem, Y=normalized_resistant,
                   calc_second_order=True, print_to_console=False)

# Get scores by type 
Si_filter = {k:Si[k] for k in ['ST', 'ST_conf', 'S1', 'S1_conf']}

# Create a DataFrame out of them
Si_df = pd.DataFrame(Si_filter, index=problem['names'])

# Get indices and error bars
indices = Si_df[['S1','ST']]
err = Si_df[['S1_conf','ST_conf']]

In [ ]:
# Plot the results
fig, ax = plt.subplots(1)
indices.plot.bar(yerr=err.values.T,ax=ax)
fig.set_size_inches(8,6)
fig.subplots_adjust(bottom=0.3)

In [ ]:
sns.set_style('whitegrid')
fig = plot_sobol_indices(Si, problem, criterion='ST', threshold=0.005)
fig.set_size_inches(7,7)
plt.show()